In [61]:
# --- CELL 1: DEFINITIVE PARSER (Final Refactored Version) ---

import os
import re
import shutil

# --- Stage 1: Load and Combine Section Files ---
print("--- STAGE 1: LOADING FILES ---")
INPUT_SECTIONS_DIR = "extracted_sections" 
e1ap_asn1_spec = ""
if not os.path.exists(INPUT_SECTIONS_DIR):
    print(f"FATAL ERROR: Directory '{INPUT_SECTIONS_DIR}' not found.")
else:
    section_files = sorted([f for f in os.listdir(INPUT_SECTIONS_DIR) if f.endswith('.txt')])
    if not section_files: print(f"FATAL ERROR: No '.txt' files found.")
    else:
        all_content = [];
        for filename in section_files:
            filepath = os.path.join(INPUT_SECTIONS_DIR, filename)
            with open(filepath, 'r', encoding='utf-8') as f: all_content.append(f.read())
        e1ap_asn1_spec = "\n".join(all_content)
        print(f"Successfully loaded and combined {len(section_files)} section files.")

# --- Stage 2: Parsing ---
print("\n--- STAGE 2: PARSING DEFINITIONS ---")

# (Helper functions are correct and remain the same)
def parse_constraints(line_part):
    constraints = {'min': None, 'max': None, 'ext': "..." in line_part}
    match = re.search(r"\((?:SIZE\s*\()?\s*([\w-]+)(?:\s*\.\.\s*([\w-]+))?\s*\)?\)", line_part)
    if match: constraints['min'] = match.group(1); constraints['max'] = match.group(2) or match.group(1)
    return constraints
def parse_presence(line_part):
    if "OPTIONAL" in line_part: return "optional"
    if "CONDITIONAL" in line_part: return "conditional"
    return "mandatory"
def standard_string(input_string):
    if not input_string: return ""
    input_string = re.sub(r"^(id-)", "", input_string)
    return "".join([word.capitalize() for word in re.split(r"[-_.&{}\s]", input_string) if word])

# --- Main Parser ---
DEFINITIONS = {}
lines = e1ap_asn1_spec.splitlines()
i = 0
while i < len(lines):
    line = lines[i].strip()
    if not line or line.startswith("--"): i += 1; continue
    
    # --- NEW REFACTORED PARSING LOGIC ---
    # We use a flat `if/elif` structure inspired by the mentor's script.
    
    # Case 1: Handle SEQUENCE and CHOICE blocks
    if "::= SEQUENCE" in line and line.endswith("{") or "::= CHOICE" in line and line.endswith("{"):
        name = line.split("::=")[0].strip()
        item = {"name": name, "ies": []}
        if "SEQUENCE" in line: item["type"] = "SEQUENCE"
        else: item["type"] = "CHOICE"
        
        block_content = []; j = i + 1; nest_level = 1
        while j < len(lines):
            line_in_block = lines[j].strip();
            if "{" in line_in_block: nest_level += 1
            if "}" in line_in_block: nest_level -= 1
            if nest_level == 0: break
            block_content.append(line_in_block); j += 1
        i = j
        item["ext"] = "..." in "".join(block_content)
        for member_line in block_content:
            if not member_line or member_line.startswith("--") or "..." in member_line: continue
            parts = member_line.split();
            if len(parts) < 2: continue
            member_details = { "ie": parts[0].replace(',', ''), "type": parts[1].replace(',', ''), "presence": parse_presence(member_line) }
            member_details.update(parse_constraints(member_line)); item["ies"].append(member_details)
        DEFINITIONS[name] = item

    # Case 2: Handle all other definitions containing `::=`
    elif "::=" in line:
        full_line = line
        if i + 1 < len(lines) and lines[i+1].strip().startswith('('):
            full_line += " " + lines[i+1].strip(); i += 1
            
        name = full_line.split("::=")[0].strip()
        def_part = full_line.split("::=", 1)[1].strip()
        item = {"name": name}
        
        # Now determine the type from the definition part
        if "SEQUENCE OF" in def_part or "SEQUENCE" in def_part and "OF" in def_part:
            item["type"] = "LIST"; match = re.search(r'OF\s+([\w-]+)', def_part);
            if match: item["item_type"] = match.group(1)
            item.update(parse_constraints(def_part))
        elif "ENUMERATED" in def_part:
            item["type"] = "ENUMERATED"; match = re.search(r'\{(.*)\}', full_line);
            if match: item["list"] = [v.strip() for v in match.group(1).replace("...", "").split(',') if v.strip()]
        elif "INTEGER" in full_line: # Check the full line for the keyword
            item["type"] = "INTEGER"
            # Handle the three cases for INTEGER constraints
            val_assign_match = re.search(r'::=\s*([\d]+)', full_line)
            if val_assign_match:
                value = val_assign_match.group(1)
                item['min'] = value; item['max'] = value
            elif "{" in def_part:
                constraint_part_match = re.search(r'\}\s*(\(.*\))', def_part)
                if constraint_part_match: item.update(parse_constraints(constraint_part_match.group(1)))
            else:
                item.update(parse_constraints(def_part))
        # ... (add BITSTRING, OCTETSTRING here if needed)

        if "type" in item:
            DEFINITIONS[name] = item
    i += 1

print(f"Parsing complete. Found {len(DEFINITIONS)} definitions.")

# --- Stage 3: Type Resolution and Data Inspection ---
# (This part is correct and remains the same)
print("\n--- STAGE 3: RESOLVING & INSPECTING ---")
resolved_count = 0; unresolved_types = set()
for definition_name, definition_body in DEFINITIONS.items():
    if definition_body.get("type") in ["SEQUENCE", "CHOICE"]:
        for member in definition_body.get("ies", []):
            member_type_name = member.get("type")
            if member_type_name in DEFINITIONS:
                resolved_count += 1; referenced_definition = DEFINITIONS[member_type_name]
                member['base_type'] = referenced_definition.get("type")
                if member.get('min') is None: member['min'] = referenced_definition.get('min')
                if member.get('max') is None: member['max'] = referenced_definition.get('max')
                if member.get('ext') is None: member['ext'] = referenced_definition.get('ext')
            known_base_types = {"INTEGER", "BITSTRING", "BIT", "OCTETSTRING", "OCTET", "ENUMERATED", "BOOLEAN", "NULL", "SEQUENCE"}
            if member_type_name not in known_base_types and member_type_name not in DEFINITIONS:
                 unresolved_types.add(member_type_name)
print(f"Type resolution complete. Linked {resolved_count} custom type references.")
print("\n--- DATA INSPECTION: PARSED INTEGER CONSTRAINTS ---")
for name, item in sorted(DEFINITIONS.items()):
    if item.get("type") == "INTEGER":
        min_val = item.get('min', 'None'); max_val = item.get('max', 'None')
        print(f"  - {name:<35}: min = {min_val}, max = {max_val}")

--- STAGE 1: LOADING FILES ---
Successfully loaded and combined 6 section files.

--- STAGE 2: PARSING DEFINITIONS ---
Parsing complete. Found 401 definitions.

--- STAGE 3: RESOLVING & INSPECTING ---
Type resolution complete. Linked 339 custom type references.

--- DATA INSPECTION: PARSED INTEGER CONSTRAINTS ---
  - AveragingWindow                    : min = None, max = None
  - BitRate                            : min = None, max = None
  - Cell-Group-ID                      : min = None, max = None
  - DRB-ID                             : min = None, max = None
  - ExtendedPacketDelayBudget          : min = None, max = None
  - GNB-CU-CP-UE-E1AP-ID               : min = 0, max = 4294967295
  - GNB-CU-UP-Capacity                 : min = 0, max = 255
  - GNB-CU-UP-ID                       : min = 0, max = 68719476735
  - GNB-CU-UP-UE-E1AP-ID               : min = 0, max = 4294967295
  - GNB-DU-ID                          : min = 0, max = 68719476735
  - HFN                            

In [62]:
# --- CELL 2: PARSER FOR SPECIAL CONSTANTS (ProcedureCode & ProtocolIE-ID) ---

# This cell specifically finds and parses ProcedureCode and ProtocolIE-ID
# definitions and adds them to the main DEFINITIONS dictionary.

import re

print("\n--- PARSING SPECIAL CONSTANTS ---")

if 'DEFINITIONS' not in locals() or 'e1ap_asn1_spec' not in locals():
    print("\nFATAL ERROR: 'DEFINITIONS' or 'e1ap_asn1_spec' not found.")
    print("Please run the main Parser cell (Cell 1) before running this cell.")
else:
    found_special_constants = 0
    # We use the raw lines from the spec loaded in Cell 1
    lines = e1ap_asn1_spec.splitlines()

    for line in lines:
        line = line.strip()
        if not line or "::=" not in line:
            continue

        # Check for the specific keywords we are targeting
        if "ProcedureCode ::=" in line or "ProtocolIE-ID ::=" in line:
            name_part, value_part = line.split("::=", 1)
            
            # The name is the first word on the left side
            name = name_part.strip().split()[0]
            # The value is the number on the right side
            value = value_part.strip()
            
            if name and value.isdigit():
                # Determine the type based on the keyword
                item_type = ""
                if "ProcedureCode" in line:
                    item_type = "ProcedureCode"
                elif "ProtocolIE-ID" in line:
                    item_type = "ProtocolIE-ID"
                
                # Add this new definition to our main dictionary
                DEFINITIONS[name] = {
                    'name': name,
                    'type': item_type,
                    'min': value,
                    'max': value
                }
                found_special_constants += 1

    print(f"  - SUCCESS: Found and added {found_special_constants} ProcedureCode/ProtocolIE-ID definitions.")

print("\nSpecial constant parsing complete.")


--- PARSING SPECIAL CONSTANTS ---
  - SUCCESS: Found and added 153 ProcedureCode/ProtocolIE-ID definitions.

Special constant parsing complete.


In [63]:
# --- CELL 3: MANUAL FIXER ---

# This cell performs manual corrections on specific definitions
# that are known to be difficult for the generic parser.

print("\n--- APPLYING MANUAL FIXES FOR PROBLEMATIC IEs ---")

if 'DEFINITIONS' not in locals():
    print("\nFATAL ERROR: The 'DEFINITIONS' dictionary was not found.")
    print("Please run the Parser cell (Cell 1) before running this cell.")
else:
    # --- Fix for PriorityLevel ---
    priority_level_name = "PriorityLevel"
    if priority_level_name in DEFINITIONS:
        print(f"  - Found '{priority_level_name}'. Overwriting its constraints.")
        DEFINITIONS[priority_level_name]['min'] = '0'
        DEFINITIONS[priority_level_name]['max'] = '15'
        print(f"    -> New constraints: min = 0, max = 15")
    else:
        print(f"  - WARNING: '{priority_level_name}' not found. Fix was not applied.")

    print("\nManual fixes applied.")


--- APPLYING MANUAL FIXES FOR PROBLEMATIC IEs ---
  - Found 'PriorityLevel'. Overwriting its constraints.
    -> New constraints: min = 0, max = 15

Manual fixes applied.


In [64]:
# --- CELL 2: THE CONFLICT RESOLVER (Corrected Logic) ---

print("\n--- RESOLVING CONSTANT vs. TYPE CONFLICTS ---")

CONSTANT_NAMES = []
INTEGER_TYPE_NAMES = []
# (We can add lists for ENUMERATED_TYPE_NAMES, etc. here later)

if 'DEFINITIONS' not in locals():
    print("\nFATAL ERROR: The 'DEFINITIONS' dictionary was not found.")
else:
    # Manual Fix for PriorityLevel
    if "PriorityLevel" in DEFINITIONS:
        DEFINITIONS["PriorityLevel"]['min'] = '0'
        DEFINITIONS["PriorityLevel"]['max'] = '15'
        
    # --- THIS IS THE FINAL FIX ---
    # We loop through ALL definitions to find constants.
    for name, item in sorted(DEFINITIONS.items()):
        min_val = item.get('min')
        max_val = item.get('max')
        
        # This is the universal rule for a constant in our parsed data.
        is_constant = (min_val is not None and min_val == max_val and min_val.isdigit())
        
        if is_constant:
            # If it's a constant, add its name to the list.
            CONSTANT_NAMES.append(name)
        
        # If it's NOT a constant, then we check if it's an INTEGER type.
        elif item.get("type") == "INTEGER":
            INTEGER_TYPE_NAMES.append(name)
    # --- END FIX ---

    print(f"  - Resolution complete.")
    print(f"  - Identified {len(CONSTANT_NAMES)} definitions as Constants.")
    print(f"  - Identified {len(INTEGER_TYPE_NAMES)} definitions as INTEGER Types.")

print("\nConflict resolution step complete.")


--- RESOLVING CONSTANT vs. TYPE CONFLICTS ---
  - Resolution complete.
  - Identified 176 definitions as Constants.
  - Identified 33 definitions as INTEGER Types.

Conflict resolution step complete.


In [65]:
# --- CELL: DEFINITIVE CONSTANT GENERATOR (with Sorting) ---
import os, shutil, re
OUTPUT_DIR = "e1ap_ies"
if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)
def standard_string(input_string):
    if not input_string: return ""
    input_string = re.sub(r"^(id-)", "", input_string)
    return "".join([word.capitalize() for word in re.split(r"[-_.&{}\s:]", input_string) if word])

print("\n--- GENERATING ALL CONSTANT DEFINITIONS ---")
integer_consts = []; proc_code_consts = []; protocol_ie_consts = []
if 'DEFINITIONS' not in locals() or 'CONSTANT_NAMES' not in locals():
    print("\nFATAL ERROR: Prerequisite data not found.")
else:
    for name in CONSTANT_NAMES:
        item = DEFINITIONS[name]
        const_asn1_name = item['name'].split()[0]
        base_name = standard_string(const_asn1_name)
        if not base_name: continue
        const_value = item.get('min')
        item_type = item.get("type", "")
        if item_type == "ProcedureCode":
            final_const_name = f"ProcedureCode_{base_name}"
            proc_code_consts.append((final_const_name, const_value))
        elif item_type == "ProtocolIE-ID":
            final_const_name = f"ProtocolIEID_{base_name}"
            protocol_ie_consts.append((final_const_name, const_value))
        elif item_type == "INTEGER":
            final_const_name = base_name
            integer_consts.append((final_const_name, const_value))

# --- THIS IS THE FINAL FIX ---
# Sort each list of constants by its value (the second element of the tuple).
# We must convert the value to an integer for correct numerical sorting.
def get_sort_key(item):
    try:
        # Try to convert the value to an integer for sorting
        return int(item[1])
    except (ValueError, TypeError):
        # If it fails (e.g., value is None or a string like 'max...'),
        # return a large number to sort it last.
        return float('inf')

integer_consts.sort(key=get_sort_key)
proc_code_consts.sort(key=get_sort_key)
protocol_ie_consts.sort(key=get_sort_key)
# --- END FIX ---

def format_const_block(constants_list, typed=False):
    if not constants_list: return ""
    max_name_len = 0
    if constants_list: max_name_len = max(len(name) for name, value in constants_list)
    formatted_lines = []
    for name, value in constants_list:
        padding1 = " " * (max_name_len - len(name))
        if typed: formatted_lines.append(f"\t{name}{padding1} int64 = {value}")
        else: formatted_lines.append(f"\t{name}{padding1} = {value}")
    return "\n".join(formatted_lines) + "\n"

# Assemble the final Go file content
final_go_code = "// This file contains constant values extracted from the ASN.1 specification.\n\n"
if integer_consts:
    final_go_code += "// General INTEGER Constants\nconst (\n" + format_const_block(integer_consts, typed=True) + ")\n\n"
if proc_code_consts:
    final_go_code += "// Procedure Codes\nconst (\n" + format_const_block(proc_code_consts) + ")\n\n"
if protocol_ie_consts:
    final_go_code += "// Protocol IE IDs\nconst (\n" + format_const_block(protocol_ie_consts) + ")\n\n"
    
total_found = len(integer_consts) + len(proc_code_consts) + len(protocol_ie_consts)
if total_found > 0:
    file_path = os.path.join(OUTPUT_DIR, "e1ap_constants.go")
    with open(file_path, 'w', encoding='utf-8') as f: f.write(f"package e1ap_ies\n\n{final_go_code}")
    print(f"  - SUCCESS: Wrote {total_found} total constants to 'e1ap_constants.go'.")
else: print("  - No constants found to generate.")
print("\nConstant generation step complete.")


--- GENERATING ALL CONSTANT DEFINITIONS ---
  - SUCCESS: Wrote 176 total constants to 'e1ap_constants.go'.

Constant generation step complete.


In [66]:
# --- CELL 4: INTEGER TYPE GENERATOR ---
import os, shutil, re
OUTPUT_DIR = "e1ap_ies"
def standard_string(input_string):
    if not input_string: return ""
    return "".join([word.capitalize() for word in re.split(r"[-_.&{}\s:]", input_string.replace('INTEGER','').strip()) if word])
print("\n--- GENERATING INTEGER TYPES ---")
integer_definitions_content = ""
if 'DEFINITIONS' not in locals() or 'INTEGER_TYPE_NAMES' not in locals():
    print("\nFATAL ERROR: Prerequisite data not found. Run Cell 1 and 2 first.")
else:
    for name in INTEGER_TYPE_NAMES:
        item = DEFINITIONS[name]
        go_name = standard_string(name)
        if "{" in name or ":" in name or not go_name: continue
        const_lines = []
        min_val = item.get('min'); max_val = item.get('max')
        if min_val is not None:
            min_val_str = standard_string(min_val) if not min_val.isdigit() else min_val
            const_lines.append(f"\t{go_name}MinValue = {min_val_str}")
        if max_val is not None:
            max_val_str = standard_string(max_val) if not max_val.isdigit() else max_val
            const_lines.append(f"\t{go_name}MaxValue = {max_val_str}")
        integer_definitions_content += f"type {go_name} int64\n"
        if const_lines:
            integer_definitions_content += "const (\n" + "\n".join(const_lines) + "\n)\n"
        integer_definitions_content += "\n"
    if integer_definitions_content:
        file_path = os.path.join(OUTPUT_DIR, "integer_types.go")
        with open(file_path, 'w', encoding='utf-8') as f: f.write("package e1ap_ies\n\n" + integer_definitions_content)
        print(f"Code generation complete. Wrote {len(INTEGER_TYPE_NAMES)} INTEGER type definitions.")
    else: print("No INTEGER type aliases were found to generate.")


--- GENERATING INTEGER TYPES ---
Code generation complete. Wrote 33 INTEGER type definitions.
